In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, explode, sequence, to_date,col,lit,concat, unix_timestamp, from_unixtime, date_format,year,month,dayofmonth,sum,max

from pyspark.conf import SparkConf

In [2]:
import warnings
warnings.simplefilter('ignore')


In [3]:
spark=SparkSession\
.builder\
.master("local[1]")\
.appName("Spark API")\
.config("spark.jars", "/home/kafka2/documents/orderManagement/postgresql-42.6.0.jar") \
.config("hive.metastore.uris", "thrift://192.168.1.12:9083")\
.config("spark.sql.warehouse.dir","hdfs://192.168.1.5:9000/user/hive/warehouse")\
.config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
.enableHiveSupport()\
.getOrCreate()



24/02/20 16:25:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

# Extract POSTGRES to Data Lake

## Connect to POSTGRES

In [ ]:
table_names_list  = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://192.168.1.6:5432/postgres") \
    .option("dbtable", "information_schema.tables") \
    .option("user", "postgres") \
    .option("password", "Nintendo1") \
    .option("driver", "org.postgresql.Driver") \
    .load()\
    .filter("table_schema = 'public'").select("table_name")

In [ ]:
table_names_list = [row.table_name for row in table_names_list.collect()]
table_names_list

In [ ]:
from datetime import date
today = str(date.today())
tempDate=today.split("-")
tempDate
year=tempDate[0]
month=tempDate[1]
day=tempDate[2]

In [ ]:
tableName="order_product_entity"


### Extract data into data lake

In [ ]:
sc = spark.sparkContext
sc.setLogLevel("WARN")
jvm = sc._jvm
conf = sc._jsc.hadoopConfiguration()
tableLocation = f"hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/{tableName}"
uri = jvm.java.net.URI(tableLocation)
fs = jvm.org.apache.hadoop.fs.FileSystem.get(uri, conf)
exists=fs.exists(jvm.org.apache.hadoop.fs.Path(tableLocation))
exists

In [ ]:
tableLocation=f'hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/{tableName}'

In [ ]:
if exists:
    df= spark.read.parquet(tableLocation)
    maxCurrent=df.agg({"id": "max"}).collect()[0][0]
    tbQuery=f'(select * from {tableName} where id > {maxCurrent}) as table1'
else:
    tbQuery=f'(select * from {tableName} ) as table1'

In [ ]:
tableDB = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://192.168.1.6:5432/postgres") \
    .option("dbtable", tbQuery) \
    .option("user", "postgres") \
    .option("password", "Nintendo1") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [ ]:
tableDB.show()

In [ ]:
outputDF=tableDB.withColumn("year",lit(year)).withColumn("month",lit(month)).withColumn("day",lit(day))

In [ ]:
outputDF.show()

In [ ]:
cols = len(empDF.columns)
print(f"DataFrame Columns count : {cols}")

In [ ]:
rows=outputDF.count()
rows

- Check new row

In [ ]:
if rows>0:
    outputDF.write.partitionBy('year', 'month','day').mode('append').parquet(tableLocation)

### Create database in Hive

In [ ]:
spark.sql("create database if not exists order")

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS employee ( eid int, name String,salary String, destination String)")

In [5]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|   buffvn|
|  default|
+---------+



In [7]:
spark.sql("use buffvn")

DataFrame[]

In [8]:
spark.sql("show tables").show()

+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|   buffvn|customer_entity_dim|      false|
|   buffvn|          dates_dim|      false|
|   buffvn|   order_entity_dim|      false|
|   buffvn|         order_fact|      false|
|   buffvn| product_entity_dim|      false|
+---------+-------------------+-----------+



In [ ]:
spark.sql("select * from summary_daily").show()

In [ ]:
spark.sql("drop database order CASCADE  ")

In [9]:
spark.sql("drop table order_fact ")

DataFrame[]

## READ table from Hive



In [10]:
order_entity=spark.read.parquet("hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/order_entity").drop("year","month","day")
order_product_entity=spark.read.parquet("hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/order_product_entity").drop("year","month","day")
order_entity_order_products_order_product_entity=spark.read.parquet("hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/order_entity_order_products_order_product_entity").drop("year","month","day")

##  Create Fact Table

 Handle the same coulunm name  `createdAt` in `order_product_entity order_entity` table

- Rename createdAt column in order_product_entity table

In [11]:
order_product_entity=order_product_entity.withColumnRenamed("createdAt","createdAtProduct")
order_product_entity=order_product_entity.withColumnRenamed("updatedAt","updatedAtProduct")
order_entity=order_entity.withColumnRenamed("id","idOrder")
order_entity=order_entity.withColumnRenamed("status","statusOrder")

In [12]:
order_entity.printSchema()

root
 |-- idOrder: integer (nullable = true)
 |-- orderCode: string (nullable = true)
 |-- guestId: integer (nullable = true)
 |-- totalPrice: decimal(38,18) (nullable = true)
 |-- shippingAddress: string (nullable = true)
 |-- note: string (nullable = true)
 |-- shippingFee: integer (nullable = true)
 |-- discountCode: string (nullable = true)
 |-- discountFee: decimal(38,18) (nullable = true)
 |-- shippingMethod: string (nullable = true)
 |-- shippingProvinceId: string (nullable = true)
 |-- shippingDistrictId: string (nullable = true)
 |-- shippingWardId: string (nullable = true)
 |-- paymentMethod: string (nullable = true)
 |-- statusOrder: string (nullable = true)
 |-- createdAt: timestamp (nullable = true)
 |-- updatedAt: timestamp (nullable = true)



- Create date column interger type

In [13]:
temp2=order_entity_order_products_order_product_entity.join(order_entity,order_entity_order_products_order_product_entity["orderEntityId"] == order_entity["idOrder"],"inner")


In [14]:
temp3=temp2.join(order_product_entity,temp2["orderProductEntityId"] == order_product_entity["id"],"inner")


In [15]:
newdf = temp3.withColumn("orderDateKey",(year(temp3["createdAt"])*10000)+(month(temp3["createdAt"])*100)+(dayofmonth(temp3["createdAt"])))


In [19]:
groupList=["orderDateKey","orderEntityId","guestId","orderProductEntityId","quantity","currentPrice","productId","totalPrice"]

In [20]:
# outputDF=newdf.groupBy(groupList).agg(sum("currentPrice").alias("currentPrice"),sum( "quantity").alias("sumQuantity"))


In [21]:
outputDF=newdf.select(groupList)

- Set date execution

In [22]:
year="2024"
month="02"
day="20"

In [23]:
outputDF=outputDF.withColumn("year",lit(year)).withColumn("month",lit(month)).withColumn("day",lit(day))

In [24]:
outputDF.show(2)

+------------+-------------+-------+--------------------+--------+--------------------+---------+--------------------+----+-----+---+
|orderDateKey|orderEntityId|guestId|orderProductEntityId|quantity|        currentPrice|productId|          totalPrice|year|month|day|
+------------+-------------+-------+--------------------+--------+--------------------+---------+--------------------+----+-----+---+
|    20220829|           41|      3|                  54|       4|10000.00000000000...|        9|184100.0000000000...|2024|   02| 20|
|    20220829|           42|      3|                  55|       4|10000.00000000000...|        9|184100.0000000000...|2024|   02| 20|
+------------+-------------+-------+--------------------+--------+--------------------+---------+--------------------+----+-----+---+
only showing top 2 rows



- Write fact table to DW

In [25]:
outputDF.write.partitionBy('year', 'month','day').mode('append').saveAsTable("buffvn.order_fact")

24/02/20 16:39:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### Create  Order dimention table


In [19]:
order_entity_dim=order_entity.withColumn("year",lit(year)).withColumn("month",lit(month)).withColumn("day",lit(day))

In [20]:
order_entity_dim.write.partitionBy('year', 'month','day').mode('append').saveAsTable("buffvn.order_entity_dim")

### Create Customer dimention


In [21]:
customer_entity=spark.read.parquet("hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/customer_entity").drop("year","month","day")

In [22]:
customer_entity_dim=customer_entity.withColumn("year",lit(year)).withColumn("month",lit(month)).withColumn("day",lit(day))

In [23]:
customer_entity_dim.write.partitionBy('year', 'month','day').mode('append').saveAsTable("buffvn.customer_entity_dim")

### Create Pruduction dimention


In [24]:
product_entity=spark.read.parquet("hdfs://192.168.1.5:9000/user/hive/datalake/buff-vn/product_entity").drop("year","month","day")

In [25]:
product_entity_dim=product_entity.withColumn("year",lit(year)).withColumn("month",lit(month)).withColumn("day",lit(day))

In [26]:
product_entity_dim.write.partitionBy('year', 'month','day').mode('append').saveAsTable("buffvn.product_entity_dim")

### Create Date Dimention

In [27]:
beginDate = '2022-01-01'
endDate = '2024-12-31'

In [28]:
spark.sql(f"select explode(sequence(to_date('{beginDate}'), to_date('{endDate}'), interval 1 day)) as calendarDate").createOrReplaceTempView('dates')

In [29]:
spark.sql("""select
  year(calendarDate) * 10000 + month(calendarDate) * 100 + day(calendarDate) as dateInt,
  CalendarDate,
  year(calendarDate) AS CalendarYear,
  date_format(calendarDate, 'MMMM') as CalendarMonth,
  month(calendarDate) as MonthOfYear,
  date_format(calendarDate, 'EEEE') as CalendarDay,
  dayofweek(calendarDate) AS DayOfWeek,
  weekday(calendarDate) + 1 as DayOfWeekStartMonday,
  case
    when weekday(calendarDate) < 5 then 'Y' else 'N' end as IsWeekDay, dayofmonth(calendarDate) as DayOfMonth, case when calendarDate = last_day(calendarDate) then 'Y' else 'N' end as IsLastDayOfMonth, dayofyear(calendarDate) as DayOfYear, weekofyear(calendarDate) as WeekOfYearIso, quarter(calendarDate) as QuarterOfYear, /* Use fiscal periods needed by organization fiscal calendar */ case when month(calendarDate) >= 10 then year(calendarDate) + 1
    else year(calendarDate)
  end as FiscalYearOctToSep,
  (month(calendarDate) + 2) % 12 + 1 AS FiscalMonthOctToSep,
  case
    when month(calendarDate) >= 7 then year(calendarDate) + 1
    else year(calendarDate)
  end as FiscalYearJulToJun,
  (month(calendarDate) + 5) % 12 + 1 AS FiscalMonthJulToJun
from
  dates
order by
  calendarDate """).createOrReplaceTempView('dates')

In [31]:
dates_dim = spark.sql("SELECT * FROM dates")


- Write Date Dimention table to DW

In [33]:
dates_dim.write.mode('overwrite').saveAsTable("buffvn.dates_dim")